In [1]:
import pandas as pd 
import seaborn as sns 
import plotly_express as px

In [52]:
df = pd.read_csv(r"C:\Users\talal\Downloads\housing.csv\housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
df.shape

(20640, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [8]:
px.box(df, x='total_bedrooms')

In [9]:
df['total_bedrooms'].mean()

np.float64(537.8705525375618)

In [53]:
df['total_bedrooms'] = df['total_bedrooms'].fillna(df['total_bedrooms'].median())

In [11]:
df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [12]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [14]:
px.histogram(df, x='ocean_proximity', title='Count of ocean proximity category')

In [20]:
df_ocean_proximity_price = df.groupby('ocean_proximity')['median_house_value'].mean().reset_index()
px.histogram(df_ocean_proximity_price, x='ocean_proximity', y='median_house_value', title='Median price of houses by ocean proximity category')

In [25]:
px.scatter(df, x='total_bedrooms', y='median_house_value', color='ocean_proximity',
           color_discrete_map={'INLAND': 'red',
                               '<1H OCEAN': 'yellow', 
                               'NEAR OCEAN': 'green',
                               'NAR BAY': 'gray', 
                               'ISLAND': 'blue'}, title='Relationship between Number of bedrooms and house price')

In [54]:
X = df.drop(['median_house_value', 'ocean_proximity'], axis=1)
y = df['median_house_value']

In [55]:
X_get_dummies = pd.get_dummies(df['ocean_proximity'], drop_first=True).astype(int)
X = pd.concat([X, X_get_dummies], axis=1)
X.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0,0,1,0


In [56]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(X)

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.3, random_state=42)

In [58]:
import tensorflow as tf

In [59]:
X_train.shape

(14448, 12)

In [71]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

In [72]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,593 (103.88 KB)

 Trainable params: 26,593 (103.88 KB)

 Non-trainable params: 0 (0.00 B)

In [74]:
history = model.fit(X_train, y_train, batch_size=10, validation_split=0.2, epochs=200)

Epoch 1/200


1156/1156 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3358691072.0000 - mae: 40445.7188 - val_loss: 3508605696.0000 - val_mae: 41158.6133
Epoch 2/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3157155840.0000 - mae: 39455.7266 - val_loss: 3491074048.0000 - val_mae: 41156.8750
Epoch 3/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3173970176.0000 - mae: 39388.0117 - val_loss: 3425034496.0000 - val_mae: 39949.5625
Epoch 4/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3222643200.0000 - mae: 39118.1914 - val_loss: 3365594880.0000 - val_mae: 39514.5469
Epoch 5/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3068517376.0000 - mae: 38342.3711 - val_loss: 3703293696.0000 - val_mae: 40669.7773
Epoch 6/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3037210624.0000 - mae: 38207.0703 - val_loss: 3338455808.0000 - val_mae: 39121.6094
Epoch 7/200
1156/1156 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2938865920.0000 - mae: 37469.9102 - val_loss: 3322234880.0000 - val

In [75]:
model.predict(X_test)

194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[ 87238.805],
       [292595.03 ],
       [442576.78 ],
       ...,
       [193035.14 ],
       [164228.42 ],
       [528682.6  ]], dtype=float32)

In [77]:
model.evaluate(X_test, y_test)

194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2765316608.0000 - mae: 35230.6211


[2818166016.0, 34891.62109375]